# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770858883866                   -0.53    9.0         
  2   -2.772141864913       -2.89       -1.31    1.0    194ms
  3   -2.772170177349       -4.55       -2.59    1.0    145ms
  4   -2.772170719417       -6.27       -3.88    2.0    161ms
  5   -2.772170722807       -8.47       -4.34    2.0    183ms
  6   -2.772170723011       -9.69       -5.72    1.0    146ms
  7   -2.772170723015      -11.44       -6.35    2.0    167ms
  8   -2.772170723015      -13.61       -6.65    2.0    196ms
  9   -2.772170723015      -13.81       -7.48    1.0    151ms
 10   -2.772170723015   +  -13.94       -8.63    2.0    179ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770729235734                   -0.53    8.0         
  2   -2.772048340665       -2.88       -1.31    1.0    138ms
  3   -2.7

1.773558104265066

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770657552915                   -0.53    9.0         
  2   -2.772051532579       -2.86       -1.30    1.0    169ms
  3   -2.772082830337       -4.50       -2.65    1.0    142ms
  4   -2.772083415983       -6.23       -3.96    2.0    184ms
  5   -2.772083417742       -8.75       -4.61    2.0    163ms
  6   -2.772083417808      -10.18       -5.61    1.0    146ms
  7   -2.772083417811      -11.55       -6.17    2.0    187ms
  8   -2.772083417811      -13.19       -7.04    1.0    151ms
  9   -2.772083417811   +  -13.99       -7.86    1.0    171ms
 10   -2.772083417811      -14.03       -8.72    2.0    180ms

Polarizability via ForwardDiff:       1.7725349667585004
Polarizability via finite difference: 1.773558104265066
